# hypopy tutorial

In [4]:
import numpy as np
import hypo

## Joint hypocenter-velocity inversion with P-wave arrivals only

### Create first synthetic data

Define limits and build grid

In [5]:
xmin = 100.0
xmax = 201.0
ymin = 100.0
ymax = 201.0
zmin = 0.0
zmax = 71.0

dx = 10.0

g = hypo.Grid3D(np.arange(xmin, xmax, dx), np.arange(ymin, ymax, dx), np.arange(zmin, zmax, dx))